Make sure to run `ollama serve` in WSL.  
Visit [http://localhost:11434](http://localhost:11434) to verify access from Windows.  
If the port isn't accessible from Windows, then just restart the computer.

In [3]:
import base64
from PIL import Image
import requests
from pathlib import Path
folder = Path("transcripts")
found = sorted([x for x in list(folder.glob("*.txt")) if "out" not in x.name])

prompt_path = found[-1]
outfile = folder / prompt_path.name.replace(".txt", ".out.txt")
print("Prompt path:", prompt_path.name)

def get_image_path(prompt_path: Path) -> Path:
    image_extensions = [".png", ".jpg", ".jpeg", ".gif", ".bmp", ".tiff", ".webp"]
    i, rest = prompt_path.stem.split(" ", 1)
    i = int(i)
    while i > 0:
        for ext in image_extensions:
            image_path = folder / f"{i:03} {rest}{ext}"
            if image_path.exists():
                return image_path
        i -= 1
    return None

image_path = get_image_path(prompt_path)
assert image_path is not None and image_path.exists(), f"No image found for {prompt_path.name}"
print("Image path:", image_path.name)

with open(image_path, "rb") as image_file:
    image_bytes = image_file.read()
image_b64 = base64.b64encode(image_bytes).decode("utf-8")

prompt = prompt_path.read_text()

resp = requests.post('http://localhost:11434/api/generate', json={
    "model": "llava",
    "prompt": prompt,
    "stream": False,
    "images": [image_b64],
})
text = resp.json()["response"]
outfile.write_text(text)
print(text)

Prompt path: 011 - trinitarianism.txt
Image path: 011 - trinitarianism.png
 The image shows a screenshot of a social media post. The post features a meme with a text overlay that reads "Trinitarianism is incorrect." Below the text, there are images of two religious texts: "The Church of Nicene" and "Holy Spirit and 318 Bishops."

The user interface indicates that the post has been made by someone named "Arius." The meme is likely intended to express a disagreement with the concept of Trinitarianism, which is a belief held by certain Christian denominations that there are three persons—the Father, Son, and Holy Spirit—in one Godhead.

The reaction to this post seems to be positive for Arius, as evidenced by the "God" emoji indicating approval or agreement. However, it's important to note that religious beliefs can be deeply personal and sensitive, and sharing such content may lead to different interpretations and responses from different individuals. 


In [2]:
i, rest = prompt_path.name.split(" ", 1)
i = int(i)
i += 1
new_path = folder / f"{i:03} {rest}"
import shutil
shutil.copy(prompt_path, new_path)

WindowsPath('transcripts/011 - wizard nuts.txt')